In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
import re

# Machine Learning imports for k-means
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Import CRUD module for MongoDB
import animal_shelter
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Access username and password from environment variables
username = os.getenv('MONGO_USERNAME')
password = os.getenv('MONGO_PASSWORD')

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read({}))

# Preprocess the data
def preprocess_data(df):
    # Extract features for clustering
    df_filtered = df[['breed', 'age_upon_outcome_in_weeks', 'location_lat', 'location_long']]

    # Convert categorical data (e.g., breed) to numerical data using LabelEncoder
    le = LabelEncoder()
    df_filtered['breed_encoded'] = le.fit_transform(df_filtered['breed'])

    # Drop non-encoded breed column
    df_filtered = df_filtered.drop('breed', axis=1)

    # Scale the features (k-means works better with normalized data)
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_filtered)

    return df_scaled, le

# Apply K-Means clustering
def apply_kmeans(df_scaled, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(df_scaled)

    # Return the cluster labels
    return kmeans.labels_

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('KMeansDashboard')

# Add cluster filter and dropdown for number of clusters
app.layout = html.Div([
    html.A([html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=250, width=251))]),
    html.Center(html.B(html.H1('Paasch SNHU CS-340 Dashboard'))),
    html.Hr(),

    # Radio items for filtering
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
        ],
        value='All'
    ),

    # Dropdown for selecting the number of clusters for k-means
    html.Label('Number of Clusters:'),
    dcc.Dropdown(id='cluster-dropdown', options=[{'label': i, 'value': i} for i in range(2, 11)], value=3),

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single",
        selected_rows=[],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    html.Br(),
    html.Hr(),

    # Add cluster visualization and pie chart
    html.Div(className='row', style={'display': 'flex', 'justify-content': 'center'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ]),

    html.Br(),

    # Pie Chart to show breed distribution
    html.Div([
        html.H3("Breed Distribution"),
        dcc.Graph(id='pie-chart')
    ]),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update data table, perform k-means clustering, and update visualizations
@app.callback(
    [Output('datatable-id', 'data'), Output('graph-id', 'children'), Output('pie-chart', 'figure')],
    [Input('filter-type', 'value'), Input('cluster-dropdown', 'value')]
)
def update_dashboard(filter_type, n_clusters):
    # Filter data based on the selected option
    if filter_type == 'Water':
        ds = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}})))
    elif filter_type == 'Mountain':
        ds = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog",
                               "Siberian Huskey", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}})))
    elif filter_type == 'Disaster':
        ds = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever",
                               "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}})))
    else:
        ds = pd.DataFrame.from_records(shelter.read({}))

    # Preprocess the data for clustering
    df_scaled, label_encoder = preprocess_data(ds)

    # Apply k-means clustering
    cluster_labels = apply_kmeans(df_scaled, n_clusters)

    # Add cluster labels to the dataframe
    ds['cluster'] = cluster_labels

    # Create a scatter plot to visualize the clusters
    scatter_fig = px.scatter(ds, x='location_lat', y='location_long', color='cluster',
                             hover_data=['breed', 'age_upon_outcome_in_weeks'])

    # Create a pie chart for breed distribution
    pie_chart = px.pie(ds, names='breed', title="Breed Distribution")

    # Update the data table and graphs
    data = ds.to_dict('records')

    return data, dcc.Graph(figure=scatter_fig), pie_chart

app.run_server(debug=True)


ModuleNotFoundError: No module named 'jupyter_dash'